In [17]:
from collections import namedtuple
import matplotlib.pyplot as plt
from tqdm import tqdm
import xxhash
import os

In [15]:
Result = namedtuple("Result", ["count", "maxlen_lo", "maxlen_hi", "t"])

In [26]:
MISC_SAVES = "/home/vlad/Documents/SublimeMerge/ExpoSizeStringSearch/llm_intersection_local/intersection_test_files/misc_saves"
CNT_BYTES_ATTACK = 250 * (50 << 20)

In [31]:
results = []
with open(f"{MISC_SAVES}/output4_inter_bs.txt", "rb") as fin:
    lines = list(fin.readlines())
    i = 0
    while i < len(lines):
        count, maxlen_lo, maxlen_hi = map(
            lambda x: int(x.split('=')[-1]),
            lines[i].decode("utf-8").strip().replace(' ', '').split(',')
        )
        t, len_t = [], 0
        j = i + 1
        while j < len(lines) and len_t <= maxlen_lo: # <= ptc am bonus ultimul endl.
            t.append(lines[j])
            len_t += len(lines[j])
            j += 1
        i = j + 1 # sar peste "---"
        results.append(Result(count, maxlen_lo, maxlen_hi, b''.join(t)[:-1]))

In [32]:
len(results)

131566

In [53]:
hashes = set()
cnt_gen_lo, cnt_gen_hi = 0, 0
for r in results:
    hh = xxhash.xxh64(r.t).hexdigest()
    cnt_useless = sum([r.t.count(ch) for ch in [b' ', b'-', b'.']])
    if hh not in hashes and cnt_useless < len(r.t):
        hashes.add(hh)
        cnt_gen_lo += r.count * r.maxlen_lo
        cnt_gen_hi += r.count * r.maxlen_hi

In [54]:
cnt_gen_lo, cnt_gen_hi

(71566803, 74178526)

In [57]:
cnt_gen_lo / CNT_BYTES_ATTACK * 100, cnt_gen_hi / CNT_BYTES_ATTACK * 100

(0.5460113754272461, 0.5659372406005859)